In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
sys.path.append("../")

import pandas as pd
import time
from azureml.core import Workspace
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

#### Prerequisite
If you use the provision feature, then just need a workspace ws object
If you need to create the cluster manually
    1. Create a service principal and secret (SP)
    2. Provision an ADX cluster and create a DB
    3. Add the SP to be contributor of the cluster
    4. pip install following packages: pip install --upgrade azure-mgmt-kusto azure-kusto-ingest azure-kusto-data azure-identity 



#### Provisioning resource


The following code provision ADX cluster and register neccessary information at the workspace's keyvault

In [ ]:
#Option to set parameters to a custom ADX cluster
# from monitoring import KV_SP_ID, KV_SP_KEY, KV_ADX_DB, KV_ADX_URI, KV_TENANT_ID
# tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
# #Application ID
# client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
# #Client Secret, set it at your WS' keyvault with key name same as your client_id
# client_secret = kv.get_secret(client_id)
# subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

# cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster
# kv.set_secret(KV_SP_ID,client_id)
# kv.set_secret(KV_SP_KEY,client_secret)
# kv.set_secret(KV_ADX_DB,"db01")
# kv.set_secret(KV_ADX_URI,cluster_uri)
# kv.set_secret(KV_TENANT_ID,tenant_id)


In [ ]:
from monitoring.management import provision
provision(ws)

#### Test Ingestion


In [ ]:
from monitoring.data_collector import Online_Collector
table_name = "isd_weather_test4" #new dataset

sample_pd_data = pd.read_parquet("data/test_data.parquet").head(10)
sample_pd_data['timestamp'] = sample_pd_data['datetime']
sample_pd_data.drop(['datetime'], inplace=True, axis=1)

online_collector = Online_Collector(table_name,ws=ws)


In [ ]:
online_collector.cluster_uri

In [ ]:
online_collector.stream_collect_df(sample_pd_data)

In [ ]:
online_collector.batch_collect(sample_pd_data)

### Streaming Ingestion and Real Time Visualization

In [ ]:

# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_df(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('today'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging()



In [ ]:
from monitoring.query import RT_Visualization
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='12h',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Anomaly Detection

In [ ]:
from monitoring.query import KustoQuery
query = KustoQuery(streaming_table_name, ws)

query.anomaly_detection(min_t="4/26/2022 10:08:45", max_t="4/23/2022 10:10:00", step="0.5s", metric="metric1", agg="avg", ts_col="timestamp", groupby="lr", sensitivity=0.1, filter="0.001")


### Drift Query

In [20]:
from monitoring.drift_analysis import Drift_Analysis
drift_analysis =Drift_Analysis(ws)


In [21]:
df_output = drift_analysis.analyze_drift(limit=30000000,base_table_name = 'ISDWeather',tgt_table_name='ISDWeather', base_dt_from='2013-04-13', base_dt_to='2014-05-13', tgt_dt_from='2013-04-13', tgt_dt_to='2014-05-13', bin='30d')


In [23]:
df_output

,frequency,numeric_feature,wasserstein,base_min,base_max,base_mean,target_min,target_max,target_mean,categorical_feature,euclidean,base_dcount,target_dcount
0,2014-04-08 00:00:00+00:00,latitude,1.785103,-90,87,40,-90,87,40,usaf,28.0,12320,11572
1,2014-04-08 00:00:00+00:00,latitude,1.785103,-90,87,40,-90,87,40,wban,28.0,2404,2364
2,2014-04-08 00:00:00+00:00,longitude,12.042559,-179,999,-80,-179,999,-80,usaf,28.0,12320,11572
3,2014-04-08 00:00:00+00:00,longitude,12.042559,-179,999,-80,-179,999,-80,wban,28.0,2404,2364
4,2014-04-08 00:00:00+00:00,windSpeed,0.027431,0,90,3,0,79,3,usaf,28.0,12320,11572
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2013-06-12 00:00:00+00:00,year_1,0.867188,2013,2014,2014,2013,2013,2013,wban,28.0,2404,2366
416,2013-06-12 00:00:00+00:00,day_1,5.572159,1,31,14,12,30,21,usaf,28.0,12320,11534
417,2013-06-12 00:00:00+00:00,day_1,5.572159,1,31,14,12,30,21,wban,28.0,2404,2366
418,2013-06-12 00:00:00+00:00,version_1,0.0,1,1,1,1,1,1,usaf,28.0,12320,11534


In [27]:
from jupyter_dash import JupyterDash
from datetime import date
from dateutil import parser
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import json
from monitoring.drift_analysis import Drift_Analysis
drift_analysis =Drift_Analysis(ws)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

tables_list = drift_analysis.list_tables()


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                tables_list,
                tables_list[0],
                id='tables',
            ),
            dcc.DatePickerRange(
            id='timeline',

            ),
            dcc.Store(id='intermediate-value')
        ], style={'width': '20%', 'display': 'inline-block'}),

        html.Div([ dcc.Dropdown(
                id='columns'
            ),

        ], style={'width': '20%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='graph'),


])

@app.callback(
    Output('columns', 'options'),
    Input('tables', 'value'))
def set_columns_options(table_name):
    return drift_analysis.list_table_columns(table_name)['AttributeName'].values

@app.callback(
    Output('columns', 'value'),
    Input('columns', 'options'))
def set_columns_value(available_options):
    return available_options[0]
@app.callback(
    [Output('timeline', 'min_date_allowed'),Output('timeline', 'max_date_allowed'),Output('timeline', 'initial_visible_month'),Output('timeline', 'start_date'), Output('timeline', 'end_date')],
    Input('tables', 'value'))
def set_columns_options(table_name):
    time_range =drift_analysis.get_time_range(table_name)

    start_date = parser.parse(time_range[0]).replace(microsecond=0, second=0, minute=0)
    end_date = parser.parse(time_range[1]).replace(microsecond=0, second=0, minute=0)
    
    return [start_date, end_date,end_date, start_date, end_date]
@app.callback(
    Output('intermediate-value', 'data'),
    Input('tables', 'value'),
    Input('timeline', 'start_date'),
    Input('timeline', 'end_date'),     
    )
def calculate(table_name, start_date, end_date):

    query_df = drift_analysis.analyze_drift(limit=100000,base_table_name = f'{table_name}',tgt_table_name=f'{table_name}', base_dt_from=f'{start_date}', base_dt_to=f'{end_date}', tgt_dt_from=f'{start_date}', tgt_dt_to=f'{end_date}', bin='30d')
    # output = json.dumps({"timestamp_col":timestamp_col, "data":query_df.to_json(date_format='iso', orient='split')})
    return query_df.to_json(date_format='iso', orient='split')


@app.callback(
    Output('graph', 'figure'),
    Input('columns', 'value'),
    Input('intermediate-value', 'data')


    )
def update_figure(column,jsonified_cleaned_data):
    # dff = pd.read_json(json.loads(jsonified_cleaned_data['data']), orient='split')
    dff = pd.read_json(jsonified_cleaned_data, orient='split')
    dff.sort_values("frequency", inplace=True)
    if dff[dff['numeric_feature']==column].shape[0]>0:
        y_values = dff[dff['numeric_feature']==column]['wasserstein']
        x_values= dff[dff['numeric_feature']==column]["frequency"]
    elif dff[dff['categorical_feature']==column].shape[0]>0:
        y_values = dff[dff['categorical_feature']==column]['target_dcount']

        x_values = dff[dff['categorical_feature']==column]["frequency"]
    else:
        x_values =[0]
        y_values =[0]
    print(column)
    print("x_values ", x_values)
    print("y_values ", y_values)
    fig = go.Figure([go.Scatter(x=x_values, y=y_values, name=f"{column}")])


    return fig


app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
StartTime
x_values  [0]
y_values  [0]
sales
x_values  0      2012-12-14T00:00:00.000Z
1      2012-12-14T00:00:00.000Z
2      2013-01-13T00:00:00.000Z
3      2013-01-13T00:00:00.000Z
4      2013-02-12T00:00:00.000Z
                 ...           
119    2017-10-19T00:00:00.000Z
120    2017-11-18T00:00:00.000Z
121    2017-11-18T00:00:00.000Z
122    2017-12-18T00:00:00.000Z
123    2017-12-18T00:00:00.000Z
Name: frequency, Length: 124, dtype: object
y_values  0      14.999465
1      14.999465
2      14.012798
3      14.012798
4      10.743707
         ...    
119     4.864690
120     0.743001
121     0.743001
122     5.211975
123     5.211975
Name: wasserstein, Length: 124, dtype: float64
EventId
x_values  0     2021-01-01T00:00:00.000Z
1     2021-01-31T00:00:00.000Z
2     2021-